In [1]:
!wget https://github.com/TheZarif/disaster-tweets/raw/main/nlp-getting-started.zip

--2023-11-29 05:36:41--  https://github.com/TheZarif/disaster-tweets/raw/main/nlp-getting-started.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/TheZarif/disaster-tweets/main/nlp-getting-started.zip [following]
--2023-11-29 05:36:41--  https://raw.githubusercontent.com/TheZarif/disaster-tweets/main/nlp-getting-started.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp-getting-started.zip’

nlp-getting-started 100%[===================>] 593.11K  --.-KB/s    in 0.06s   

2023-11-29 05:36:41 (10.1 MB/s) - ‘nlp-getting-started.zip’ saved [607343/

In [2]:
!unzip nlp-getting-started.zip -d data

Archive:  nlp-getting-started.zip
  inflating: data/sample_submission.csv  
  inflating: data/test.csv           
  inflating: data/train.csv          


In [3]:
import pandas as pd

df_train = pd.read_csv("/content/data/train.csv")
df_test = pd.read_csv("/content/data/test.csv")

print('Training Set Shape = {}'.format(df_train.shape))
print('Training Set Memory Usage = {:.2f} MB'.format(df_train.memory_usage().sum() / 1024**2))
print('Test Set Shape = {}'.format(df_test.shape))
print('Test Set Memory Usage = {:.2f} MB'.format(df_test.memory_usage().sum() / 1024**2))

Training Set Shape = (7613, 5)
Training Set Memory Usage = 0.29 MB
Test Set Shape = (3263, 4)
Test Set Memory Usage = 0.10 MB


In [4]:
df_train["length"] = df_train["text"].apply(lambda x : len(x))
df_test["length"] = df_test["text"].apply(lambda x : len(x))

print("Train Length Stat")
print(df_train["length"].describe())
print()

print("Test Length Stat")
print(df_test["length"].describe())

Train Length Stat
count    7613.000000
mean      101.037436
std        33.781325
min         7.000000
25%        78.000000
50%       107.000000
75%       133.000000
max       157.000000
Name: length, dtype: float64

Test Length Stat
count    3263.000000
mean      102.108183
std        33.972158
min         5.000000
25%        78.000000
50%       109.000000
75%       134.000000
max       151.000000
Name: length, dtype: float64


In [5]:
NUM_TRAINING_EXAMPLES = df_train.shape[0]
TRAIN_SPLIT = 0.8
VAL_SPLIT = 0.2
EPOCHS = 2

In [6]:
from sklearn.model_selection import train_test_split

X = df_train["text"]
y = df_train["target"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=VAL_SPLIT, random_state=42)

X_test = df_test["text"]

In [7]:
def preprocess(text):
    preprocessed_text = []
    for t in text.split():
        if len(t) > 1:
            t = '@user' if t[0] == '@' and t.count('@') == 1 else t
            t = 'http' if t.startswith('http') else t
        preprocessed_text.append(t)
    return ' '.join(preprocessed_text)

X_train = X_train.apply(preprocess)
X_val = X_val.apply(preprocess)
X_test = X_test.apply(preprocess)

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline

# Define a range of candidate max_features values
candidate_max_features = [1000, 5000, 10000, 20000]

# Create a parameter grid for GridSearchCV
param_grid = {
    'vectorizer__max_features': candidate_max_features,
    'classifier__kernel': ['linear', 'rbf'],
    'classifier__C': [0.1, 1, 10]
}

# Create a pipeline with TF-IDF vectorizer and SVM classifier
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', SVC())
])

# Create GridSearchCV with the parameter grid and pipeline
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')

# Fit GridSearchCV on your training data
grid_search.fit(X_train, y_train)

# Get the best parameters and best estimator
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

print(f"Best Parameters: {best_params}")

# Make predictions on the validation set using the best estimator
y_pred = best_estimator.predict(X_val)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy on Validation Set: {accuracy:.2f}")

Best Parameters: {'classifier__C': 1, 'classifier__kernel': 'rbf', 'vectorizer__max_features': 10000}
Accuracy on Validation Set: 0.81


In [10]:
# Use the best estimator for predictions
y_pred = grid_search.best_estimator_.predict(X_val)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy on Validation Set: {accuracy:.2f}")

Accuracy on Validation Set: 0.81


In [ ]:
test_pred = grid_search.best_estimator_.predict(X_test)

In [12]:
df_test['target'] = test_pred

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
df_test[['id', 'target']].to_csv('/content/drive/My Drive/disaster-tweets/svm.csv', index=False)